In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

from sklearn.linear_model import Lasso

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model
from riskslim.setup_functions import get_conservative_offset
from riskslim.coefficient_set import CoefficientSet
from riskslim.lattice_cpa import run_lattice_cpa
from riskslim.lattice_cpa import setup_lattice_cpa, finish_lattice_cpa

from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.utils import shuffle

### Modeling -- RiskSLIM

In [2]:
def risk_slim(data, max_coefficient, max_L0_value, c0_value, max_runtime = 120, w_pos = 1, max_offset=50):
    
    
    """
    @parameters:
    
    max_coefficient:  value of largest/smallest coefficient
    max_L0_value:     maximum model size (set as float(inf))
    max_offset:       maximum value of offset parameter (optional)
    c0_value:         L0-penalty parameter such that c0_value > 0; larger values -> 
                      sparser models; we set to a small value (1e-6) so that we get a model with max_L0_value terms
    max_runtime:      max algorithm running time
    w_pos:            relative weight on examples with y = +1; w_neg = 1.00 (optional)
    
    """
    
    # create coefficient set and set the value of the offset parameter
    coef_set = CoefficientSet(variable_names = data['variable_names'], lb = 0, ub = max_coefficient, sign = 0)
    conservative_offset = get_conservative_offset(data, coef_set, max_L0_value)
    max_offset = min(max_offset, conservative_offset)
    coef_set['(Intercept)'].ub = max_offset
    coef_set['(Intercept)'].lb = -max_offset

    constraints = {
        'L0_min': 0,
        'L0_max': max_L0_value,
        'coef_set':coef_set,
    }
    
    # Set parameters
    settings = {
        # Problem Parameters
        'c0_value': c0_value,
        'w_pos': w_pos,

        # LCPA Settings
        'max_runtime': max_runtime,                         # max runtime for LCPA
        'max_tolerance': np.finfo('float').eps,             # tolerance to stop LCPA (set to 0 to return provably optimal solution)
        'display_cplex_progress': True,                     # print CPLEX progress on screen
        'loss_computation': 'lookup',                       # how to compute the loss function ('normal','fast','lookup')
        
        # LCPA Improvements
        'round_flag': False,                                # round continuous solutions with SeqRd
        'polish_flag': False,                               # polish integer feasible solutions with DCD
        'chained_updates_flag': False,                      # use chained updates
        'add_cuts_at_heuristic_solutions': True,            # add cuts at integer feasible solutions found using polishing/rounding
        
        # Initialization
        'initialization_flag': True,                        # use initialization procedure
        'init_max_runtime': 300.0,                          # max time to run CPA in initialization procedure
        'init_max_coefficient_gap': 0.49,

        # CPLEX Solver Parameters
        'cplex_randomseed': 0,                              # random seed
        'cplex_mipemphasis': 0,                             # cplex MIP strategy
    }
    

    # train model using lattice_cpa
    model_info, mip_info, lcpa_info = run_lattice_cpa(data, constraints, settings)
        
    return model_info, mip_info, lcpa_info

In [3]:
def riskslim_prediction(X, feature_name, model_info):
    
    """
    @parameters
    
    X: test input features (np.array)
    feature_name: feature names
    model_info: output from RiskSLIM model
    
    """
    
    ## initialize parameters
    dictionary = {}
    prob = np.zeros(len(X))
    scores = np.zeros(len(X))
    
    ## prepare statistics
    subtraction_score = model_info['solution'][0]
    coefs = model_info['solution'][1:]
    index = np.where(coefs != 0)[0]
    
    nonzero_coefs = coefs[index]
    features = feature_name[index]
    X_sub = X[:,index]
    
    ## build dictionaries
    for i in range(len(features)):
        single_feature = features[i]
        coef = nonzero_coefs[i]
        dictionary.update({single_feature: coef})
        
    ## calculate probability
    for i in range(len(X_sub)):
        summation = 0
        for j in range(len(features)):
            a = X_sub[i,j]
            summation += dictionary[features[j]] * a
        scores[i] = summation
    
    prob = 1/(1+np.exp(-(scores + subtraction_score)))
    
    return prob

In [4]:
def riskslim_accuracy(X, Y, feature_name, model_info, threshold=0.5):
    
    prob = riskslim_prediction(X, feature_name, model_info)
    pred = np.mean((prob > threshold) == Y)
    
    return pred

### Lasso Feature Selection

In [5]:
## load stumps data
data = pd.read_csv("C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/KY Recidivism/KY data/kentucky_stumps.csv")
X, Y = data.loc[:,:'current_violence>=1'], data['recid_six_month'].values
cols = X.columns

In [6]:
## lasso
lasso = Lasso(random_state=816, alpha=0.001).fit(X, Y)
selected_features = cols[lasso.coef_ != 0].tolist()
len(selected_features), roc_auc_score(Y, lasso.predict(X))

(35, 0.7659345752558604)

#### subset features

In [7]:
### Subset features
selected_features.insert(0, 'recid_six_month')
sub_data = data[selected_features]
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

### Cross Validation

In [8]:
SLIM_X, SLIM_Y = sub_X.values, sub_Y.reshape(-1,1)
variable_names = sub_X.columns.tolist()
outcome_name = 'recid_six_month'
sample_weights = np.repeat(1, len(sub_Y))

In [9]:
#cv = KFold(n_splits=5, random_state=816, shuffle=True)
cv = KFold(n_splits=5, random_state=816, shuffle=True)
train_auc, test_auc = [], []

i = 0
for train, test in cv.split(SLIM_X, SLIM_Y):
    
    ## subset train data & store test data
    X_train, Y_train = SLIM_X[train], SLIM_Y[train]
    X_test, Y_test = SLIM_X[test], SLIM_Y[test]
    sample_weights_train, sample_weights_test = sample_weights[train], sample_weights[test]

    ## create new data dictionary
    new_train_data = {
        'X': X_train,
        'Y': Y_train,
        'variable_names': variable_names,
        'outcome_name': outcome_name,
        'sample_weights': sample_weights_train
    }
        
    ## fit the model
    model_info, mip_info, lcpa_info = risk_slim(new_train_data, max_coefficient=20, max_L0_value=10, 
                                                c0_value=1e-5, max_runtime=400)
    print_model(model_info['solution'], new_train_data)
    
    ## change data format
    X_train, X_test = X_train[:,1:], X_test[:,1:] ## remove the first column, which is "intercept"
    Y_train[Y_train == -1] = 0 ## change -1 to 0
    Y_test[Y_test == -1] = 0
    
    ## probability & accuracy
    train_prob = riskslim_prediction(X_train, cols, model_info).reshape(-1,1)
    test_prob = riskslim_prediction(X_test, cols, model_info).reshape(-1,1)
    
    ## AUC
    train_auc.append(roc_auc_score(Y_train, train_prob))
    test_auc.append(roc_auc_score(Y_test, test_prob))

setting c0 = 0.0 to ensure that intercept is not penalized
08/17/19 @ 04:11 PM | 1501 rows in lookup table
08/17/19 @ 04:11 PM | ------------------------------------------------------------
08/17/19 @ 04:11 PM | runnning initialization procedure
08/17/19 @ 04:11 PM | ------------------------------------------------------------
08/17/19 @ 04:11 PM | CPA produced 2 cuts
08/17/19 @ 04:11 PM | running naive rounding on 67 solutions
08/17/19 @ 04:11 PM | best objective value: 0.2553
08/17/19 @ 04:11 PM | rounding produced 5 integer solutions
08/17/19 @ 04:11 PM | best objective value is 0.2642
08/17/19 @ 04:11 PM | running sequential rounding on 67 solutions
08/17/19 @ 04:11 PM | best objective value: 0.2553
08/17/19 @ 04:11 PM | sequential rounding produced 6 integer solutions
08/17/19 @ 04:11 PM | best objective value: 0.2596
08/17/19 @ 04:11 PM | polishing 11 solutions
08/17/19 @ 04:11 PM | best objective value: 0.2596
08/17/19 @ 04:11 PM | polishing produced 5 integer solutions
08/17/19

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 04:11 PM | adding 250 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2596.
Tried aggregator 1 time.
Reduced MIP has 37 rows, 74 columns, and 143 nonzeros.
Reduced MIP has 35 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 122296 44965        cutoff              0.2596        0.2541   916289    2.12%          rho_29 D 122296 122295     31
 124523 45537        0.2588     3        0.2596        0.2541   932228    2.10%          rho_22 D 124523 124521     20
Elapsed time = 115.27 sec. (168251.31 ticks, tree = 21.25 MB, solutions = 2)
 126758 46096        0.2570    12        0.2596        0.2542   947456    2.09%          rho_21 U 126758 126757     26
 128992 46650        cutoff              0.2596        0.2542   962836    2.07%          rho_23 U 128992  14663     24
 131184 47105        cutoff              0.2596        0.2542   978229    2.05%          rho_18 D 131184 131182     22
 133416 47589        0.2595     7        0.2596        0.2543   993526    2.03%          rho_23 D 133416 133415     19
 135591 48055        0.2571    13        0.2596        0.2543  1008923    2.02%          rho_21 U 135591 135590     23
 137785 48576        cutoff              0.2596        0.2544  1024597    2.00%          r

08/17/19 @ 04:18 PM | CPA produced 2 cuts
08/17/19 @ 04:18 PM | running naive rounding on 72 solutions
08/17/19 @ 04:18 PM | best objective value: 0.2552
08/17/19 @ 04:18 PM | rounding produced 5 integer solutions
08/17/19 @ 04:18 PM | best objective value is 0.2622
08/17/19 @ 04:18 PM | running sequential rounding on 72 solutions
08/17/19 @ 04:18 PM | best objective value: 0.2552
08/17/19 @ 04:18 PM | sequential rounding produced 6 integer solutions
08/17/19 @ 04:18 PM | best objective value: 0.2599
08/17/19 @ 04:18 PM | polishing 11 solutions
08/17/19 @ 04:18 PM | best objective value: 0.2599
08/17/19 @ 04:18 PM | polishing produced 5 integer solutions
08/17/19 @ 04:18 PM | initialization produced 12 feasible solutions
08/17/19 @ 04:18 PM | best objective value: 0.2599
08/17/19 @ 04:18 PM | ------------------------------------------------------------
08/17/19 @ 04:18 PM | completed initialization procedure
08/17/19 @ 04:18 PM | --------------------------------------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 04:18 PM | adding 251 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2599.
Tried aggregator 1 time.
Reduced MIP has 37 rows, 74 columns, and 143 nonzeros.
Reduced MIP has 35 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

...

08/17/19 @ 04:38 PM | CPA produced 2 cuts
08/17/19 @ 04:38 PM | running naive rounding on 71 solutions
08/17/19 @ 04:38 PM | best objective value: 0.2540
08/17/19 @ 04:38 PM | rounding produced 5 integer solutions
08/17/19 @ 04:38 PM | best objective value is 0.2654
08/17/19 @ 04:38 PM | running sequential rounding on 71 solutions
08/17/19 @ 04:38 PM | best objective value: 0.2540
08/17/19 @ 04:38 PM | sequential rounding produced 6 integer solutions
08/17/19 @ 04:38 PM | best objective value: 0.2594
08/17/19 @ 04:38 PM | polishing 11 solutions
08/17/19 @ 04:38 PM | best objective value: 0.2594
08/17/19 @ 04:38 PM | polishing produced 4 integer solutions
08/17/19 @ 04:38 PM | initialization produced 11 feasible solutions
08/17/19 @ 04:38 PM | best objective value: 0.2594
08/17/19 @ 04:38 PM | ------------------------------------------------------------
08/17/19 @ 04:38 PM | completed initialization procedure
08/17/19 @ 04:38 PM | --------------------------------------------------------

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
08/17/19 @ 04:38 PM | adding 249 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.2594.
Tried aggregator 1 time.
Reduced MIP has 37 rows, 74 columns, and 143 nonzeros.
Reduced MIP has 35 binaries, 37 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.08 ticks)
Probing time = 0.00 sec. (0.03 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.05 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variab

 107206 43792        0.2531    18        0.2591        0.2529   787665    2.38%          rho_21 D 107206  42418     23
 109267 44433        0.2584     7        0.2591        0.2529   801085    2.37%          rho_30 U 109267 109266     29
 111264 45028        0.2542    16        0.2591        0.2530   814719    2.35%           rho_8 D 111264 111263     27
 113356 45698        0.2535    17        0.2591        0.2530   828446    2.34%          rho_12 U 113356  94918     29
Elapsed time = 83.72 sec. (168238.67 ticks, tree = 22.46 MB, solutions = 4)
 115369 46323        0.2560    19        0.2591        0.2531   841991    2.32%           rho_0 D 115369 115368     21
 117345 46906        cutoff              0.2591        0.2531   855780    2.31%          rho_34 U 117345 117344     32
 119390 47492        0.2583    11        0.2591        0.2531   869038    2.29%          rho_35 D 119390 119387     29
 121360 48058        cutoff              0.2591        0.2532   882590    2.28%          rh

 344796 75775        0.2581    18        0.2585        0.2558  2251657    1.04%          rho_23 U 344796 246506     22
 346622 75833        cutoff              0.2585        0.2558  2261855    1.03%           rho_0 D 346622  42101     22
 348429 75887        cutoff              0.2585        0.2558  2272204    1.03%          rho_19 U 348429  54152     31
 350302 75931        0.2582    11        0.2585        0.2558  2282076    1.02%          rho_32 U 350302  39893     26
 352107 75982        0.2574     9        0.2585        0.2558  2292095    1.02%          rho_20 U 352107  13674     28
 353920 76037        0.2562    14        0.2585        0.2558  2302245    1.01%          rho_16 D 353920 353919     24
 355710 76119        cutoff              0.2585        0.2559  2312516    1.01%          rho_33 U 355710 355709     27
Elapsed time = 300.03 sec. (664372.83 ticks, tree = 43.10 MB, solutions = 6)
 357488 76153        0.2581    13        0.2585        0.2559  2322593    1.00%           

  Real time             =  399.94 sec. (885541.36 ticks)
                          ------------
Total (root+branch&cut) =  400.02 sec. (885543.36 ticks)
+----------------------------------------------+------------------+-----------+
| Pr(Y = +1) = 1.0/(1.0 + exp(-(-4 + score))   |                  |           |
| ============================================ | ================ | ========= |
| p_arrest>=2                                  |         1 points |   + ..... |
| p_arrest>=4                                  |         1 points |   + ..... |
| p_arrest>=10                                 |         1 points |   + ..... |
| p_felony>=1                                  |         1 points |   + ..... |
| ============================================ | ================ | ========= |
| ADD POINTS FROM ROWS 1 to 4                  |            SCORE |   = ..... |
+----------------------------------------------+------------------+-----------+


In [10]:
np.mean(train_auc), np.mean(test_auc)

(0.7368972576860641, 0.7366660160825469)

### Save Results

In [11]:
#log model results to the model performance folder, as per standards
path = "C:\\Users\\binha\\Documents\\Duke\\Cynthia Research\\KY-analysis-mytrials\\KY Recidivism\\KY Results\\Models\\Six Month\\"                   
results = [["Model", "train_auc_mean", "train_auc_std","test_auc_mean", "test_auc_std"],
    ["Recidivism", np.mean(train_auc), np.std(train_auc), np.mean(test_auc), np.std(test_auc)]]

with open(path + 'Six Month RiskSLIM.csv', 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)